In [20]:
%load_ext autoreload
%autoreload 2

from deribit_vol_surface import get_vol_surface
import numpy as np
from pyquant.black_scholes import *
from pyquant.common import *
from pyquant.sabr import Backbone, SABRCalc
from pyquant.ssvi import SSVICalc, SSVIParams, Eta, Lambda, Alpha, Beta, Gamma_
from pyquant.svi import SVICalc
from pyquant.vol_surface import *

from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True) 
import plotly.io as pio
pio.renderers.default = 'iframe'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
vol_surface_chain_space = get_vol_surface("deribit_vol_surface.csv")
# convert to delta-space
vol_surface_delta_space: VolSurfaceDeltaSpace = SABRCalc().surface_to_delta_space(
    vol_surface_chain_space, Backbone(1.0)
)

# get all market with usable format
ssvi = SSVICalc()
calc_params, calibration_error, strikes, Ts, implied_variances = ssvi.calibrate(vol_surface_delta_space, 10, True)
ivs = np.sqrt(implied_variances/Ts)

Implied variances to calibrate to: [6.17716273e-04 4.20943471e-04 3.60679404e-04 4.23746882e-04
 6.26080351e-04 4.41702202e-02 3.16116067e-02 2.72926749e-02
 2.95834138e-02 3.85795674e-02 8.59382970e-02 6.28264898e-02
 5.45657837e-02 5.70625407e-02 7.06250790e-02 1.27890273e-01
 9.74437242e-02 8.64669946e-02 8.91139751e-02 1.06639036e-01
 1.75711784e-01 1.33604170e-01 1.19084083e-01 1.23548213e-01
 1.49841534e-01 2.21639899e-01 1.68659314e-01 1.51075140e-01
 1.57648358e-01 1.93170358e-01 2.65758877e-01 2.03207770e-01
 1.83191987e-01 1.92396409e-01 2.37753992e-01 3.10620213e-01
 2.38808471e-01 2.16775784e-01 2.29879910e-01 2.87417505e-01
 3.57139048e-01 2.75918638e-01 2.52122397e-01 2.70506788e-01
 3.43007065e-01 4.04467053e-01 3.13784788e-01 2.88382908e-01
 3.12809348e-01 4.01907680e-01]
Strikes from delta-space we calibrate to: [1760.30244267 1792.12196553 1816.71212101 1842.50241728 1876.49909563
 1420.95760002 1639.62882714 1819.54411995 2073.8125857  2385.94226778
 1306.70093152 15

In [58]:
fig = go.Figure(data=[
    go.Scatter3d(z=ivs[5:], x=strikes[5:], y=Ts[5:], mode='markers'),
    go.Surface(z=ivs[5:], x=strikes[5:], y=Ts[5:])

])
fig.update_layout(title='SSVI Volatility Surface', template="plotly_white", width=1000, height=1000)
fig.update_scenes(xaxis_title_text='Strike, USD',
                  yaxis_title_text='Time to maturity, years',
                  zaxis_title_text='Implied volatility',
                 zaxis=dict(range=[0.4, 0.8])
                 )

                    

fig.show()

In [ ]:
ssvi_params = SSVIParams(
    Eta(calc_params[0]),
    Lambda(calc_params[1]), 
    Alpha(calc_params[2]), 
    Beta(calc_params[3]), 
    Gamma_(calc_params[4])
)
ssvi._grid_implied_variances(SSVIParams)